<a href="https://colab.research.google.com/github/coddingyun/pytorch/blob/main/FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import tensorflow as tf
import tensorflow.keras
from keras.utils import np_utils
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from tensorflow.keras import backend
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# Mnist 데이터셋 불러오기
# mnist.load_data()로 한줄에 쉽게 데이터를 불러올 수 있다. # 다운로드 시간도 굉장히 빠르다
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
print(X_train.shape)
print(Y_train.shape)

(60000, 28, 28)
(60000,)


In [ ]:
#Train set 과 Test set 정리하기
#cnn model에 넣기 위해서는 배열의 형태, dimension을 맞춰 주어야 한다.
input_shape = (28, 28, 1) # 샘플 수를 제외한 입력 형태를 정의, 모델에서 첫 레이어일 때만 정의, (행, 열, 채널 수)

#input
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1) # 28 x28 사이즈, 채널 수 1: 흑백
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

X_train = X_train.astype('float32') / 255. # 모든 값을 0과 1사이로 만들어 (scailing) 학습이 원활하도록 만든다
X_test = X_test.astype('float32') / 255.

#label
Y_train = tf.keras.utils.to_categorical(Y_train) # one-hot 인코딩
Y_test = tf.keras.utils.to_categorical(Y_test)

print('X_train shape:', X_train.shape)
print('Y_train shape:', Y_train.shape)

X_train shape: (60000, 28, 28, 1)
Y_train shape: (60000, 10)


In [ ]:
#CNN 모델 만들기
model = Sequential() #순차모델
model.add(Conv2D(32, kernel_size=5, strides=1, padding='same',  #‘same’:출력 이미지 사이즈가 입력 이미지 사이즈와 동일
                 activation='relu', # 활성화 함수
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Conv2D(64, 2, activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        8256      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
dropout (Dropout)            (None, 7, 7, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                2

#Dropout

![image.png](https://user-images.githubusercontent.com/22881216/65816695-1207ea80-e23a-11e9-923b-80f6dd564986.png)

- 전체 weight를 계산에 참여시키는 것이 아니라 layer에 포함된 weight 중에서 일부만 참여시킨다.
- 즉, 학습 시 뉴런을 임의로 삭제하여 학습하는 방법
- 훈련 시에는 임의의 비율(dropout ratio) 만큼 뉴런을 삭제한다.


### Dropout의 목표
- 무작위로 dropout을 하면서 학습을 시키면 overfitting의 원인인 co-adaptation를 방지한다.
  + co-adaptation 현상?
    * 각각의 weight들이 서로 동조화 되는 현상
    * 어느 unit이 잘못 학습한 것을 다른 unit이 보완해 줄 수 있는 형태로 학습하는 것.
    * 각각의 unit이 독자적인 meaningful feature를 가지지 못하게 된다.
- Dropout을 사용하면 각각의 unit이 meaningful feature를 가지도록 학습되므로 generalization error(일반화 오차)가 낮아지는 효과를 보인다.
- 좀 더 선명한 feature(salient feature)를 얻을 수 있다.


#Flatten
![image.png](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Flp8Uy%2FbtqA58bFVeS%2Fry5z3HTrNDwCEOfqgvPubk%2Fimg.png)

- Convolution Layer나 Max Pooling Layer 를 반복적으로 거치면 주요 특징만 추출되고 추출된 주요 특징은 전결합층에 전달되어 학습된다. 
- 전결합층에 전달하기 위해 1차원 자료로 바꿔주는데 이때 사용되는 Layer.

#Dense
![image.png](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbskuBH%2FbtqA4yIEQPD%2FkIv76pJK4PBkKpvUm7iPKk%2Fimg.png)



In [ ]:
#CNN 모델 학습하기
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) # softmax output layer # metrics는 평가기준, 일반적으로 accuracy
hist = model.fit(X_train, Y_train,
                 batch_size=128,
                 epochs=20,
                 verbose=1 # 얼마나 자세히 정보를 표기할 것인가
                 )

Epoch 1/20
469/469 [==============================] - 52s 110ms/step - loss: 0.3998 - accuracy: 0.8757
Epoch 2/20
469/469 [==============================] - 51s 109ms/step - loss: 0.1407 - accuracy: 0.9580
Epoch 3/20
469/469 [==============================] - 51s 109ms/step - loss: 0.1107 - accuracy: 0.9672
Epoch 4/20
469/469 [==============================] - 51s 109ms/step - loss: 0.0932 - accuracy: 0.9721
Epoch 5/20
469/469 [==============================] - 51s 108ms/step - loss: 0.0808 - accuracy: 0.9757
Epoch 6/20
469/469 [==============================] - 51s 109ms/step - loss: 0.0726 - accuracy: 0.9787
Epoch 7/20
469/469 [==============================] - 51s 109ms/step - loss: 0.0661 - accuracy: 0.9807
Epoch 8/20
469/469 [==============================] - 51s 108ms/step - loss: 0.0618 - accuracy: 0.9805
Epoch 9/20
469/469 [==============================] - 51s 108ms/step - loss: 0.0580 - accuracy: 0.9828
Epoch 10/20
469/469 [==============================] - 51s 108ms/step - l

In [ ]:
# Test
loss_and_metrics = model.evaluate(X_test, Y_test, batch_size=32)
print('loss_and_metrics : ' + str(loss_and_metrics))

313/313 [==============================] - 3s 9ms/step - loss: 0.0227 - accuracy: 0.9931
loss_and_metrics : [0.022723237052559853, 0.9930999875068665]


- keras 너무 간편하다!